In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format

In [2]:
%matplotlib inline

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
cd = pd.read_csv('complications.csv', dtype=str).rename(columns=lambda x: x.strip())

This dataset contains complications in U.S. hospitals. Each observation represents the number of complications and the 30-day death rate for a specific complication in a specific hospital.

The columns are as follows:

In [5]:
cd.columns

Index(['Provider ID', 'Hospital Name', 'Address', 'City', 'ZIP Code',
       'County Name', 'Phone Number', 'Year', 'Measure ID', 'Measure Name',
       'Denominator', 'Score', 'Lower Estimate', 'Higher Estimate'],
      dtype='object')

| Column Name | Description|
|-------------|------------|
|Provider ID | Unique ID of the hospital |
|Hospital Name| Name of the hospital|
|Address| Address of the hospital|
|City| Name of the city the hospital is in |
|ZIP Code | Postal code of the hospital|
|County Name | Name of the county the hospital is in|
|Phone Number | Phone number of the hospital |
|Year | The year in which the complications were recorded|
|Measure ID | Code for the complication |
|Measure Name | Name of the complication|
|Denominator| Number of underlying medical procedures|
|Score | Estimate for number of serious complications |
|Lower Estimate | Lower boundary of score|
|Higher Estimate | Upper boundary of score|

A quick overview of the dataset:

In [6]:
cd.describe()

,Provider ID,Hospital Name,Address,City,ZIP Code,County Name,Phone Number,Year,Measure ID,Measure Name,Denominator,Score,Lower Estimate,Higher Estimate
count,272788,272788,272788,272788,272788,272626,272788,272788,272788,272788,272788,272788,272788,272788
unique,4746,5119,4915,2874,4368,1526,5028,6,26,26,23771,6373,8690,11699
top,500054,MEMORIAL HOSPITAL,100 HOSPITAL DRIVE,CHICAGO,76104,LOS ANGELES,6104022273,2013,MORT_30_PN,Pneumonia (PN) 30-Day Mortality Rate,26.0,0.06,0.0,0.06
freq,89,710,378,1957,353,5421,178,58668,24942,24942,1435,2947,23907,2897


A sample of the dataset:

In [7]:
cd.sample(5, random_state=1234).head()

,Provider ID,Hospital Name,Address,City,ZIP Code,County Name,Phone Number,Year,Measure ID,Measure Name,Denominator,Score,Lower Estimate,Higher Estimate
22678,050222,SHARP CHULA VISTA MEDICAL CENTER,751 MEDICAL CENTER COURT,CHULA VISTA,91911,SAN DIEGO,6195025800,2016,MORT_30_PN,Pneumonia (PN) 30-Day Mortality Rate,474.0,14.3,12.1,16.8
22088,050191,ST MARY MEDICAL CENTER,1050 LINDEN AVE,LONG BEACH,90813,LOS ANGELES,5624919000,2012,PSI_14_POSTOP_DEHIS,A wound that splits open after surgery on the...,102.0,1.78,0.0,3.64
111295,201309,MAYO REGIONAL HOSPITAL,897 WEST MAIN STREET,DOVER FOXCROFT,04426,PISCATAQUIS,2075644251,2014,MORT_30_STK,Death rate for stroke patients,43.0,14.1,10.2,18.9
44865,100012,LEE MEMORIAL HOSPITAL,2776 CLEVELAND AVE,FORT MYERS,33901,LEE,2393321111,2012,MORT_30_PN,Pneumonia (PN) 30-Day Mortality Rate,971.0,10.6,8.9,12.5
208974,390194,BLUE MOUNTAIN HOSPITAL-GNADEN HUETTEN CAMPUS,211 NORTH 12TH STREET,LEHIGHTON,18235,CARBON,6073771300,2016,MORT_30_STK,Death rate for stroke patients,72.0,15.7,11.7,20.8


# Transform relevant columns (1 point)

All columns in the dataset are of type `str` (to preserve the ZIP code information). For **five** columns, str is the wrong type. Change the columns and explain why you changed which column.

In [8]:
cd['Year'] = pd.to_datetime(cd['Year'])
cd['Denominator'] = pd.to_numeric(cd['Denominator'])
cd['Score'] = pd.to_numeric(cd['Score'])
cd['Lower Estimate'] = pd.to_numeric(cd['Lower Estimate'])
cd['Higher Estimate'] = pd.to_numeric(cd['Higher Estimate'])

# Create a ratio between the score and the denominator (1 point)

Create a new column in the dataset that contains the ratio value.

In [9]:
cd['Ratio'] = cd['Score']/cd['Denominator']

# Create an overview of the ratio for deathly complications (2 points)

0. **Be careful when you identify 'deathly' complications and make sure that you catch all of them.**
1. Create the overview (Make sure that the overview is understandable to everyone.)
2. Choose an appropriate measure to aggregate the data. **Ignore that the data is for multiple years**.
2. Sort with the highest ratio on top.

In [10]:
cd[cd['Measure Name']\
.str.contains(r'Death|Mort')]\
.groupby(['Measure Name'])[['Ratio']]\
.mean()\
.sort_values(by='Ratio',ascending=False)

,Ratio
Measure Name,
Deaths among Patients with Serious Treatable Complications after Surgery,0.5244
Death rate for stroke patients,0.1848
Acute Myocardial Infarction (AMI) 30-Day Mortality Rate,0.1599
Heart failure (HF) 30-Day Mortality Rate,0.1217
Pneumonia (PN) 30-Day Mortality Rate,0.1159
Death rate for chronic obstructive pulmonary disease (COPD) patients,0.0833
Death rate for CABG,0.0415


# Identify the five safest hospitals (2 points)

Return a **list** with the hospitals that have, on average, the lowest rate of deadly complications. **Be careful when you identify 'deathly' complications.** (Again, ignore that the dataset covers multiple years).

In [11]:
cd[cd['Measure Name']\
.str.contains(r'Dea|Mort')]\
.groupby('Hospital Name')['Ratio']\
.mean()\
.nsmallest(5)

Hospital Name
UNIVERITY OF MD BALTO WASHINGTON  MEDICAL CENTER   0.0206
NEW YORK-PRESBYTERIAN HOSPITAL                     0.0220
NORTON HOSPITALS, INC                              0.0226
SAN JUAN VA MEDICAL CENTER                         0.0240
FLORIDA HOSPITAL                                   0.0256
Name: Ratio, dtype: float64

# Augment the datset with information about the state (1 points)

Read the file 'geo-data.csv'. **Make sure that you read all columns as string.**

In [12]:
zip_codes = pd.read_csv('geo-data.csv', dtype=str)

This file contains information about the relationship of zip codes, counties, and U.S. states. 

In [13]:
zip_codes.columns

Index(['state_fips', 'state', 'state_abbr', 'zipcode', 'county', 'city'], dtype='object')

| Column Name | Description|
|-------------|------------|
|state_fips | FIPS code for the state |
|state| Name of the state|
|state_abbr| Abbreviation of the state|
|zipcode | Postal code|
|county | County name|
|city | City name  |

1. Combine the dataset on complications with the dateset on zip codes.
2. Explain why you chose the column that used to combine the datasets.
3. **Do not worry, not all states will be matched.**

In [14]:
cds=pd.merge(cd, zip_codes, left_on='ZIP Code', right_on='zipcode', how='left' )

In [15]:
cds.columns

Index(['Provider ID', 'Hospital Name', 'Address', 'City', 'ZIP Code',
       'County Name', 'Phone Number', 'Year', 'Measure ID', 'Measure Name',
       'Denominator', 'Score', 'Lower Estimate', 'Higher Estimate', 'Ratio',
       'state_fips', 'state', 'state_abbr', 'zipcode', 'county', 'city'],
      dtype='object')

# Create an overview of the complications per U.S. state (1 point)

* Return a sorted list with the overall complication rate per state. 
* Ignore that the data is for multiple years. 
* Focus on the **best** ten states.

In [16]:
cds.groupby('state')['Ratio'].mean().nsmallest(10)

state
Delaware               0.0634
Maryland               0.0685
Florida                0.0756
District of columbia   0.0777
Virginia               0.0796
Pennsylvania           0.0869
New york               0.0881
South carolina         0.0893
North carolina         0.0899
Ohio                   0.0918
Name: Ratio, dtype: float64

# Create an overview of the change across U.S. states (2 points)

* Focus on the deathly complications only.
* Calculate the change as follows: **(average rate for last year in dataset - average rate for first year in dataset)/(average rate for first year in dataset)**

In [17]:
improved = cds[cds['Measure Name']\
.str.contains(r'Death|Mort')].groupby(['state','Year'])['Ratio'].mean().groupby(['state']).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[0]).sort_values()

In [18]:
improved

state
Delaware               -0.1874
South carolina         -0.1660
Nevada                 -0.1612
Arizona                -0.1380
Oregon                 -0.1280
Maryland               -0.1023
Alaska                 -0.0912
Tennessee              -0.0843
Alabama                -0.0736
Virginia               -0.0683
Kentucky               -0.0650
Missouri               -0.0613
Georgia                -0.0600
Utah                   -0.0572
Florida                -0.0557
Pennsylvania           -0.0550
Texas                  -0.0458
Indiana                -0.0454
Ohio                   -0.0434
West virginia          -0.0366
Louisiana              -0.0344
Washington             -0.0270
Illinois               -0.0257
Wisconsin              -0.0244
Michigan               -0.0158
North carolina         -0.0157
Hawaii                 -0.0112
Idaho                  -0.0080
Oklahoma               -0.0009
California             -0.0006
Minnesota               0.0052
Arkansas                0.0072
Mi

Which state has improved the most?

In [ ]:
improved.nsmallest(1)

state
Delaware   -0.1874
Name: Ratio, dtype: float64

Which state has the worst development?

In [20]:
improved.nlargest(1)

state
Wyoming   0.1061
Name: Ratio, dtype: float64

# Bonus (1 points)

Which state has the worst development for 'Deaths among Patients with Serious Treatable Complications after Surgery'?

In [21]:
cds.groupby(['state','Measure ID','Year'])['Ratio'].mean().groupby(['state','Measure ID']).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[0]).unstack()['PSI_4_SURG_COMP'].nlargest(10)

state
Utah                    0.2155
District of columbia    0.0553
Alaska                  0.0441
Arkansas                0.0328
Michigan                0.0075
Maryland                0.0000
Idaho                  -0.0252
Minnesota              -0.0451
Missouri               -0.0454
Tennessee              -0.0467
Name: PSI_4_SURG_COMP, dtype: float64